In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import json
import jsonlines
import re

In [21]:
pe_data=pd.read_csv("data/base_juguete - person.tsv", sep='\t')
la_data=pd.read_csv("data/base_juguete - idioma.tsv", sep='\t')
ed_data=pd.read_csv("data/base_juguete - Educacion.tsv", sep='\t')
ha_data=pd.read_csv("data/base_juguete - Habilidades.tsv", sep='\t')
tr_data=pd.read_csv("data/base_juguete - Trabajo.tsv", sep='\t')

In [22]:
####PURAS ENTIDADES

class persona(object):
    uid=""
    nombre=""
    

class idiomas(object):
    uid=""
    nombre=""
    idioma_nivel=""
    
    def __init__(self, nombre, idioma_nivel):
        self.nombre = nombre
        self.idioma_nivel = idioma_nivel
        
    def __json__(self):
        return {'idioma':self.nombre, 'idioma_nivel':self.idioma_nivel}

class habilidades(object):
    tipo=""
    descripcion=""
    
    def __init__(self, tipo, descripcion):
        self.tipo = tipo
        self.descripcion = descripcion
        
    def __json__(self):
        return {'tipo':self.tipo, 'descripcion':self.descripcion}


class educacion(object):
    uid=""
    institucion=""
    area=""
    fecha_inicio=""
    fecha_fin=""
    titulo=""
    extra_curriculares=""
    
    def __init__(self,institucion, area, fecha_inicio, fecha_fin, titulo):        
        self.institucion=institucion
        self.area=area
        self.fecha_inicio=fecha_inicio
        self.fecha_fin=fecha_fin
        self.titulo=titulo
        #self.extra_curriculares=extra_curriculares
        
    def __json__(self):
        return {'institucion':self.institucion, 'area':self.area, 'fecha_inicio':self.fecha_inicio,
                'fecha_fin':self.fecha_fin, 'titulo':self.titulo}
        
class trabajos(object):
    uid=""
    titulo_posicion=""
    resumen=""
    fecha_inicio=""
    fecha_fin=""
    actual=""
    empresa=""
    area=""
    tipo=""
    industria=""
    voluntariado=""
    
    def __init__(self, titulo_posicion, empresa, area, tipo, industria, voluntariado):
        self.titulo_posicion=titulo_posicion
        #self.resumen=resumen
        #self.fecha_inicio=fecha_inicio
        #self.fecha_fin=fecha_fin
        #self.actual=actual
        self.empresa=empresa
        self.area=area
        self.tipo=tipo
        self.industria=industria
        self.voluntariado=voluntariado
        
    def __json__(self):
        return {'titulo_posicion': self.titulo_posicion, 'empresa': self.empresa, 
                'area':self.area, 'tipo':self.tipo, 'industria':self.industria,
                'voluntariado':self.voluntariado}
        
class talento(object):
    uid=''
    nombre=''
    trabajos=[]
    cursos = []
    educacion = []
    certificaciones = []
    habilidades = []
    idiomas = []
    patentes = []
    publicaciones = []    
    
    def __init__(self, uid, nombre):
        self.uid = uid
        self.nombre = nombre
        
    def __json__(self):
        return {'uid': self.uid, 'nombre': self.nombre, 'trabajos':self.trabajos, 
                'educacion':self.educacion, 'habilidades':self.habilidades,
               'idiomas':self.idiomas}
    

class MyEncoder(json.JSONEncoder):
    """
    JSONEncoder subclass that leverages an object's `__json__()` method,
    if available, to obtain its default JSON representation. 

    """
    def default(self, obj):
        if hasattr(obj, '__json__'):
            return obj.__json__()
        return json.JSONEncoder.default(self, obj)
        
    

In [23]:
'''
HELPERS : 
'''
def parse_doc_list_plain(doc):        
    '''Preprocesing method, this code dont remove the character with graphic accent.'''
    red = ur'[^,|:a-z\u00E7\u00F1\u00E1\u00E9\u00ED\u00F3\u00FA\u00E0\u00E8\u00EC\u00F2\u00F9\u00E4\u00EB\u00EF\u00F6\u00FC\u00E2\u00EA\u00EE\u00F4\u00FB ]'
    ##sandbox    
    
    art_vect = re.sub(r' +',' ', re.sub(red,'',re.sub(ur'\u201c',' ',re.sub('\n',' ',doc).lower()))) # Remove the most of the symbols. 
    art_vect = re.sub(', ',',',art_vect) # ,\s\+   remove space after comma, it could be improved
    art_vect = re.sub(r' ','_',art_vect) # Replace the space by underscore.
    art_vect = re.sub(r',$','',art_vect)  # Remove the last comma character. just in case.
    return art_vect

In [24]:
test_text = u'La mansion puprpura de Zimbagüe es la casa, que hemos creado para tí, donde,...'

In [25]:
parse_doc_list_plain(test_text)

u'la_mansion_puprpura_de_zimbag\xfce_es_la_casa,que_hemos_creado_para_t\xed,donde'

In [31]:
talentos_data = []

for pe in pe_data.itertuples():
    data = talento(pe.uid,pe.nombre)
    match_la = la_data['uid'] == data.uid
    match_ed = ed_data['uid'] == data.uid
    match_ha = ha_data['uid'] == data.uid
    match_tr = tr_data['uid'] == data.uid    
    data.idiomas = [ idiomas(nombre = lad.idioma, idioma_nivel=lad.idioma_nivel) for lad in la_data[match_la].itertuples()]
    data.educacion = [educacion(institucion = edu.institucion, area = edu.area_estudio, titulo = edu.titulo, 
                         fecha_inicio = edu.inicio, fecha_fin = edu.fin) for edu in ed_data[match_ed].itertuples()]
    data.habilidades = [habilidades(tipo=hab.tipo, descripcion=hab.habilidades) for hab in ha_data[match_ha].itertuples() ]
    data.trabajos = [trabajos(titulo_posicion=tra.posicion,empresa=tra.empresa, 
                              area=tra.area, tipo=tra.tipo, voluntariado = tra.voluntariado, industria = tra.industria ) for tra in tr_data[match_tr].itertuples()]
    talentos_data.append(json.dumps(data,cls=MyEncoder))


### PASO 0 : Preprocesamiento

In [30]:
jlArray = []
for item in jsonlines.Reader(talentos_data).iter():
    preproc = {}
    preproc['educacion'] = parse_doc_list_plain('|'.join([u'{0}:{1}'.format(cur[u'institucion'],cur[u'titulo']) for cur in item['educacion']]))
    preproc['habilidades'] = parse_doc_list_plain('|'.join([u'{0}:{1}'.format(cur[u'tipo'],cur[u'descripcion']) for cur in item['habilidades']]))
    preproc[u'idiomas'] = parse_doc_list_plain('|'.join([u'{0}:{1}'.format(cur[u'idioma_nivel'],cur[u'idioma']) for cur in item[u'idiomas']]))
    preproc[u'trabajos'] = parse_doc_list_plain('|'.join([u'{0}:{1}:{2}'.format(cur[u'empresa'],cur[u'industria'],cur[u'titulo_posicion']) for cur in item[u'trabajos']]))
    preproc[u'uid'] = item[u'uid']
    jlArray.append(preproc)

In [32]:
dfper = pd.DataFrame(jlArray)

In [33]:
for idx in dfper.educacion.iteritems():
    print (idx)

(0, u'uanl:lic_en_econom\xeda|itesm:ms_en_computaci\xf3n|universidad_nacional_autonoma_de_mexico:doctor_en_ciencias')
(1, u'unam:lic_en_econom\xeda|itam:ms_en_ciencia_de_datos|stanford:phd_computer_science')


### PASO 1 : HOMOLOGACION 1

### PASO 2: DEDUPLICACION

### PASO 3: LIMPIEZA

### PASO 4: HOMOLOGACION 2

### PASO 5: DEDUPLICACION

### PASO 6: CLUSTERING